In [19]:
# Import 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn import svm
from lightgbm import LGBMClassifier
from sklearn.utils.class_weight import compute_sample_weight
from datetime import date
from sklearn.impute import SimpleImputer

In [20]:
pd.set_option('display.max_columns', None)

In [21]:
# Read CSV
df = pd.read_csv(r'C:\Users\Renan Muniz\Online_Env\REPO_TEST\ONLINE_DS_THEBRIDGE_RENAN_MUNIZ\ML_project\data\raw\Matches.csv')

C:\Users\Renan Muniz\AppData\Local\Temp\ipykernel_34440\634850314.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\Renan Muniz\Online_Env\REPO_TEST\ONLINE_DS_THEBRIDGE_RENAN_MUNIZ\ML_project\data\raw\Matches.csv')


In [22]:
df = df[df['Division'].isin(['SP1','E0','D1', 'I1', 'F1'])]

In [23]:
df.drop(columns=['MatchTime'], inplace= True )
df = df.fillna(0) 

In [24]:
le = LabelEncoder()
df['Results'] = le.fit_transform(df['FTResult'])

In [25]:

df['MatchDate'] = pd.to_datetime(df['MatchDate'])
df['Year'] = df['MatchDate'].dt.year 
df['Month'] = df['MatchDate'].dt.month 
df['Day'] = df['MatchDate'].dt.day 
df['DayOfWeek'] = df['MatchDate'].dt.dayofweek 
df['IsWeekend'] = df['DayOfWeek'].isin([5, 6]).astype(int)

df['EloDifference'] = df['HomeElo'] - df['AwayElo']

df['ImpliedProbHome'] = 1 / df['OddHome'] 
df['ImpliedProbDraw'] = 1 / df['OddDraw'] 
df['ImpliedProbAway'] = 1 / df['OddAway'] 

df['Home_odds'] = 1 / (df['ImpliedProbHome'] + df['ImpliedProbDraw']) 
df['Away_odds'] = 1/ (df['ImpliedProbAway'] + df['ImpliedProbDraw']) 
df['Home_Away_Odds'] = 1 / (df['ImpliedProbHome'] + df['ImpliedProbAway'])

df['Form3Difference'] = df['Form3Home'] - df['Form3Away'] 
df['Form5Difference'] = df['Form5Home'] - df['Form5Away'] 

df['ImpliedProbTotal'] = df['ImpliedProbHome']  + df['ImpliedProbAway'] 

df['BookmakerMargin'] = df['ImpliedProbTotal'] - 1 


df2 = df.sort_values(['HomeTeam', 'MatchDate'])
df3 = df.sort_values(['AwayTeam', 'MatchDate'])

df['GF3Home'] = df2.groupby('HomeTeam')['FTHome'].shift(1).rolling(3).sum().reindex(df.index)
df['GF3Away'] = df3.groupby('AwayTeam')['FTAway'].shift(1).rolling(3).sum().reindex(df.index)


df['GA3Home'] = df2.groupby('HomeTeam')['FTAway'].shift(1).rolling(3).sum().reindex(df.index)
df['GA3Away'] = df3.groupby('AwayTeam')['FTHome'].shift(1).rolling(3).sum().reindex(df.index)


df['GF5Home'] =  df2.groupby('HomeTeam')['FTHome'].shift(1).rolling(5).sum().reindex(df.index)
df['GF5Away'] = df3.groupby('AwayTeam')['FTAway'].shift(1).rolling(5).sum().reindex(df.index)

df['GA5Home'] = df2.groupby('HomeTeam')['FTAway'].shift(1).rolling(5).sum().reindex(df.index)
df['GA5Away'] = df3.groupby('AwayTeam')['FTHome'].shift(1).rolling(5).sum().reindex(df.index)

df[['GF3Away', 'GF3Home', 'GF5Away', 'GF5Home','GA5Away', 'GA5Home', 'GA3Home', 'GA3Away']] = df[['GF3Away', 'GF3Home', 'GF5Away', 'GF5Home','GA5Away', 'GA5Home', 'GA3Home', 'GA3Away']].fillna(0) 


winshome = df2.groupby('HomeTeam')['Results'].transform(
    lambda x: x.eq(2).astype(int)
                    .groupby((x != 2).cumsum())  
                    .cumsum()
                    .shift(1)
).fillna(0)
df['WinStreakHome'] = winshome.reindex(df.index)


winsaway = df3.groupby('AwayTeam')['Results'].transform(
    lambda x: x.eq(0).astype(int)
                    .groupby((x != 0).cumsum())  
                    .cumsum()
                    .shift(1)
).fillna(0)
df['WinStreakAway'] = winsaway.reindex(df.index)



drawhome = df2.groupby('HomeTeam')['Results'].transform(
    lambda x: x.eq(1).astype(int)
                    .groupby((x != 1).cumsum())  
                    .cumsum()
                    .shift(1)
).fillna(0)
df['DrawStreakHome'] = drawhome.reindex(df.index)


drawaway = df3.groupby('AwayTeam')['Results'].transform(
    lambda x: x.eq(1).astype(int)
                    .groupby((x != 1).cumsum())  
                    .cumsum()
                    .shift(1)
).fillna(0)
df['DrawStreakAway'] = drawaway.reindex(df.index)




defeathome = df2.groupby('HomeTeam')['Results'].transform(
    lambda x: x.eq(0).astype(int)
                    .groupby((x != 0).cumsum())  
                    .cumsum()
                    .shift(1)
).fillna(0)
df['DefeatStreakHome'] = defeathome.reindex(df.index)


defeataway = df3.groupby('AwayTeam')['Results'].transform(
    lambda x: x.eq(2).astype(int)
                    .groupby((x != 2).cumsum())  
                    .cumsum()
                    .shift(1)
).fillna(0)
df['DefeatStreakAway'] = defeataway.reindex(df.index)


df4 = df.sort_values(['HomeTeam', 'AwayTeam', 'MatchDate'])
h2hhome =  df4.groupby(['HomeTeam', 'AwayTeam'])['Results'].transform(lambda x: (x.shift(1) == 2).rolling(5, min_periods=1).sum())
df['H2HHomeWins'] = h2hhome.reindex(df.index)

h2haway =  df4.groupby(['AwayTeam','HomeTeam'])['Results'].transform(lambda x: (x.shift(1) == 0).rolling(5, min_periods=1).sum())
df['H2HAwayWins'] = h2haway.reindex(df.index)


emahome = df2.groupby('HomeTeam')['FTHome'].shift(1).ewm(span=3).mean()
# Exponential moving average (span=3) of goals scored by home team in previous matches (shifted).
df['GF_EMA3_Home'] = emahome.reindex(df.index)

emaaway = df3.groupby('AwayTeam')['FTAway'].shift(1).ewm(span=3).mean()
# Exponential moving average (span=3) of goals scored by away team in previous matches (shifted).
df['GF_EMA3_Away'] = emaaway.reindex(df.index)


goalhome = df2.groupby('HomeTeam')['FTHome'].shift(1).rolling(3).std()
# Standard deviation of goals scored by home team in last 3 home matches.
df['GF3HomeSTD'] = goalhome.reindex(df.index).fillna(0)

goalaway = df3.groupby('AwayTeam')['FTAway'].shift(1).rolling(3).std()
# Standard deviation of goals scored by away team in last 3 away matches.
df['GF3AwaySTD'] = goalaway.reindex(df.index).fillna(0)

points_home = {2:3, 1:1, 0:0}
df['PointsHome'] = df['Results'].map(points_home)

points_away = {0:3, 1:1, 2:0}
df['PointsAway'] = df['Results'].map(points_away)

df['Season'] = np.where(df['Month'] >= 8, df['Year'], df['Year'] - 1)

acumhome = df.groupby(['HomeTeam', 'Season'])['PointsHome'].transform(lambda x: x.shift(1).cumsum())
df['PointsAcumHome'] = acumhome.reindex(df.index).fillna(0)

acumaway = df.groupby(['AwayTeam', 'Season'])['PointsAway'].transform(lambda x: x.shift(1).cumsum())
df['PointsAcumAway'] = acumaway.reindex(df.index).fillna(0)


df2 = df.sort_values(['HomeTeam', 'MatchDate'])
df3 = df.sort_values(['AwayTeam', 'MatchDate'])
gf_home = df2.groupby(['HomeTeam', 'Season'])['FTHome'].transform(lambda x: x.shift(1).cumsum())
df['GF_Total_Home'] = gf_home.reindex(df.index).fillna(0)

gf_away = df3.groupby(['AwayTeam', 'Season'])['FTAway'].transform(lambda x: x.shift(1).cumsum())
df['GF_Total_Away'] = gf_away.reindex(df.index).fillna(0)

gf_home = df2.groupby(['HomeTeam', 'Season'])['FTAway'].transform(lambda x: x.shift(1).cumsum())
df['GA_Total_Home'] = gf_home.reindex(df.index).fillna(0)

gf_away = df3.groupby(['AwayTeam', 'Season'])['FTHome'].transform(lambda x: x.shift(1).cumsum())
df['GA_Total_Away'] = gf_away.reindex(df.index).fillna(0)

df['GD_total_Home'] = df['GF_Total_Home'] - df['GA_Total_Home']
df['GD_total_Away'] = df['GF_Total_Away'] - df['GA_Total_Away']


df['GameCount'] = 1
game_played_home = df.groupby(['HomeTeam', 'Season'])['GameCount'].transform(lambda x: x.shift(1).cumsum())
game_played_away = df.groupby(['AwayTeam', 'Season'])['GameCount'].transform(lambda x: x.shift(1).cumsum())

df['PointMeanHome'] = (df['PointsAcumHome'] / game_played_home).fillna(0)
df['PointMeanAway'] = (df['PointsAcumAway'] / game_played_away).fillna(0)

df['ScoredGoalsMeanHome']= (df['GF_Total_Home'] / game_played_home).fillna(0)
df['ScoredGoalsMeanAway']= (df['GF_Total_Away'] / game_played_away).fillna(0)

df['ConcededGoalsMeanHome']= (df['GA_Total_Home']/ game_played_home).fillna(0)
df['ConcededGoalsMeanAway']= (df['GA_Total_Away'] / game_played_away).fillna(0)

df['GoalsDifferenceMeanHome']= (df['GD_total_Home']/ game_played_home).fillna(0)
df['GoalsDifferenceMeanAway']= (df['GD_total_Away'] / game_played_away).fillna(0)

df['WinHome'] = (df['FTResult'] == 'H').astype(int)
df['Draw'] = (df['FTResult'] == 'D').astype(int)
df['WinAway'] = (df['FTResult'] == 'A').astype(int)

df['WinHomeAcum'] = df.groupby(['HomeTeam', 'Season'])['WinHome'].transform(lambda x: x.shift(1).cumsum()).fillna(0)
df['WinAwayAcum'] = df.groupby(['AwayTeam', 'Season'])['WinAway'].transform(lambda x: x.shift(1).cumsum()).fillna(0)

df['DrawHomeAcum'] = df.groupby(['HomeTeam', 'Season'])['Draw'].transform(lambda x: x.shift(1).cumsum()).fillna(0)
df['DrawAwayAcum'] = df.groupby(['AwayTeam', 'Season'])['Draw'].transform(lambda x: x.shift(1).cumsum()).fillna(0)

df['LossHomeAcum'] = df['WinAwayAcum'].fillna(0)
df['LossAwayAcum'] = df['WinHomeAcum'] .fillna(0)


df['WinRateHome'] = (df['WinHomeAcum']  / game_played_home).fillna(0)
df['WinRateAway'] = (df['WinAwayAcum'] / game_played_away).fillna(0)

df['DrawRateHome'] = (df['DrawHomeAcum'] / game_played_home).fillna(0)
df['DrawRateAway'] = (df['DrawAwayAcum'] / game_played_away).fillna(0)

df['LossRateHome'] = (df['LossHomeAcum'] / game_played_home).fillna(0)
df['LossRateAway'] = (df['LossAwayAcum'] / game_played_away).fillna(0)



df["OddsDifference"] = df["ImpliedProbHome"] - df["ImpliedProbAway"]

df["EloRatio"] = df["HomeElo"] / (df["AwayElo"] + 1e-6)

df["FormRatio"] = df["Form3Home"] / (df["Form3Away"] + 1)

df["GoalRateRatio"] = df["ScoredGoalsMeanHome"] / (df["ScoredGoalsMeanAway"] + 1)

df["WinRateDiff"] = df["WinRateHome"] - df["WinRateAway"]

df["PointsDiff"] = df["PointsAcumHome"] - df["PointsAcumAway"]

df["FormDiff"] = df["Form5Home"] - df["Form5Away"]


df["Elo_ProbDiff"] = (df["ImpliedProbHome"] - df["ImpliedProbAway"]) * df["EloDifference"]
df["FormOddsDiff"] = df["Form5Difference"] * (df["ImpliedProbHome"] - df["ImpliedProbAway"])
df["StreakDiff"] = df["WinStreakHome"] - df["DefeatStreakAway"]
df["BookieBiasHome"] = df["ImpliedProbHome"] - df["WinRateHome"]

df["BookieBiasAway"] = df["ImpliedProbAway"] - df["WinRateAway"]
df["DayOfWeek_sin"] = np.sin(2 * np.pi * df["DayOfWeek"] / 7)
df["DayOfWeek_cos"] = np.cos(2 * np.pi * df["DayOfWeek"] / 7)

df["Month_sin"] = np.sin(2 * np.pi * df["Month"] / 12)
df["Month_cos"] = np.cos(2 * np.pi * df["Month"] / 12)

df["Day_sin"] = np.sin(2 * np.pi * df["Day"] / 31)
df["Day_cos"] = np.cos(2 * np.pi * df["Day"] / 31)


df['EloOddsGap'] = (df['ImpliedProbHome'] - df['ImpliedProbAway']) - (df['EloRatio'] / (1+df['EloRatio']))

df['FormVolatility'] = (df['Form5Home'] - df['Form3Home']) - (df['Form5Away'] - df['Form3Away'])

df['OddSkew'] = (df['OddHome'] - df['OddAway']) / (df['OddHome'] + df['OddAway'])

In [26]:

df = df[df['MatchDate'] > '2010-08']


counts_home = df['HomeTeam'].value_counts()
counts_away = df['AwayTeam'].value_counts()

valid_teams = counts_home[counts_home >= 20].index.intersection(counts_away[counts_away >= 20].index)
df = df[(df['HomeTeam'].isin(valid_teams)) & (df['AwayTeam'].isin(valid_teams))]


X = df[['Division','MatchDate', 'HomeTeam', 'AwayTeam', 'Year',  'IsWeekend',
        'C_LTH', 'C_LTA', 'C_VHD', 'C_VAD', 'C_HTB', 'C_PHB',
    'HomeElo', 'AwayElo', 'EloDifference', 'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away', 'Form3Difference', 'Form5Difference',
    'GF3Home', 'GF3Away', 'GA3Home', 'GA3Away', 'GF5Home', 'GF5Away', 'GA5Home', 'GA5Away',
    'OddHome', 'OddDraw', 'OddAway', 'ImpliedProbHome', 'ImpliedProbDraw', 'ImpliedProbAway', 'BookmakerMargin',
    'HandiSize', 'HandiHome', 'HandiAway', 
    'MaxHome', 'MaxDraw', 'MaxAway', 'Over25', 'Under25', 'MaxOver25', 'MaxUnder25', 
    'WinStreakHome', 'WinStreakAway', 'DrawStreakHome', 'DrawStreakAway', 'DefeatStreakHome', 'DefeatStreakAway', 'H2HHomeWins', 'H2HAwayWins', 'GF_EMA3_Home','GF_EMA3_Away', 'GF3HomeSTD',
    'GF3AwaySTD', 'Season', 'PointsAcumHome', 'PointsAcumAway', 
    'GF_Total_Home', 'GF_Total_Away', 'GA_Total_Home', 'GA_Total_Away', 
    'GD_total_Home', 'GD_total_Away', 'PointMeanHome', 'PointMeanAway', 'ScoredGoalsMeanHome', 'ScoredGoalsMeanAway', 'ConcededGoalsMeanHome', 'ConcededGoalsMeanAway',
    'GoalsDifferenceMeanHome','GoalsDifferenceMeanAway','WinHomeAcum','WinAwayAcum','DrawHomeAcum','DrawAwayAcum','LossHomeAcum', 'LossAwayAcum', 'WinRateHome','WinRateAway','DrawRateHome',
    'DrawRateAway','LossRateHome','LossRateAway', 
    "OddsDifference", "EloRatio", "FormRatio", "GoalRateRatio", "WinRateDiff", "PointsDiff", "FormDiff", "Elo_ProbDiff", "FormOddsDiff", "StreakDiff", "BookieBiasHome", "DayOfWeek_sin", "DayOfWeek_cos", "Month_sin", "Month_cos", "Day_sin", "Day_cos",
    'OddSkew', 'FormVolatility', 'EloOddsGap', 'ImpliedProbTotal', 'BookieBiasAway'
]]
y = df['Results']
    


cols_draw = ['DrawStreakHome', 'DrawStreakAway', 
            'DrawHomeAcum', 'DrawAwayAcum',
            'DrawRateHome', 'DrawRateAway',
            'OddDraw', 'ImpliedProbDraw', 'MaxDraw']
# Dropping Draws
not_draw= y != 1
X_not_draw = X[not_draw].copy()
X_not_draw = X_not_draw.drop(columns=cols_draw)
y_not_draw = y[not_draw].copy()

In [27]:
X_train_nd = X_not_draw[X_not_draw['MatchDate'] < '2021-07'].drop(columns='MatchDate')
X_test_nd = X_not_draw[X_not_draw['MatchDate'] > '2021-08'].drop(columns='MatchDate')
y_train_nd = y_not_draw.loc[X_train_nd.index]
y_test_nd = y_not_draw.loc[X_test_nd.index]


In [34]:
t = df[df['HomeTeam'] == 'Arsenal']
t[df['FTResult'] == 'A']

C:\Users\Renan Muniz\AppData\Local\Temp\ipykernel_34440\3592974941.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  t[df['FTResult'] == 'A']


,Division,MatchDate,HomeTeam,AwayTeam,HomeElo,AwayElo,Form3Home,Form5Home,Form3Away,Form5Away,FTHome,FTAway,FTResult,HTHome,HTAway,HTResult,HomeShots,AwayShots,HomeTarget,AwayTarget,HomeFouls,AwayFouls,HomeCorners,AwayCorners,HomeYellow,AwayYellow,HomeRed,AwayRed,OddHome,OddDraw,OddAway,MaxHome,MaxDraw,MaxAway,Over25,Under25,MaxOver25,MaxUnder25,HandiSize,HandiHome,HandiAway,C_LTH,C_LTA,C_VHD,C_VAD,C_HTB,C_PHB,Results,Year,Month,Day,DayOfWeek,IsWeekend,EloDifference,ImpliedProbHome,ImpliedProbDraw,ImpliedProbAway,Home_odds,Away_odds,Home_Away_Odds,Form3Difference,Form5Difference,ImpliedProbTotal,BookmakerMargin,GF3Home,GF3Away,GA3Home,GA3Away,GF5Home,GF5Away,GA5Home,GA5Away,WinStreakHome,WinStreakAway,DrawStreakHome,DrawStreakAway,DefeatStreakHome,DefeatStreakAway,H2HHomeWins,H2HAwayWins,GF_EMA3_Home,GF_EMA3_Away,GF3HomeSTD,GF3AwaySTD,PointsHome,PointsAway,Season,PointsAcumHome,PointsAcumAway,GF_Total_Home,GF_Total_Away,GA_Total_Home,GA_Total_Away,GD_total_Home,GD_total_Away,GameCount,PointMeanHome,PointMeanAway,ScoredGoalsMeanHome,ScoredGoalsMeanAway,ConcededGoalsMeanHome,ConcededGoalsMeanAway,GoalsDifferenceMeanHome,GoalsDifferenceMeanAway,WinHome,Draw,WinAway,WinHomeAcum,WinAwayAcum,DrawHomeAcum,DrawAwayAcum,LossHomeAcum,LossAwayAcum,WinRateHome,WinRateAway,DrawRateHome,DrawRateAway,LossRateHome,LossRateAway,OddsDifference,EloRatio,FormRatio,GoalRateRatio,WinRateDiff,PointsDiff,FormDiff,Elo_ProbDiff,FormOddsDiff,StreakDiff,BookieBiasHome,BookieBiasAway,DayOfWeek_sin,DayOfWeek_cos,Month_sin,Month_cos,Day_sin,Day_cos,EloOddsGap,FormVolatility,OddSkew
63536,E0,2010-09-25,Arsenal,West Brom,1880.34,1632.23,7.0,11.0,4.0,7.0,2.0,3.0,A,0.0,0.0,D,24.0,7.0,12.0,5.0,15.0,14.0,16.0,4.0,4.0,3.0,0.0,0.0,1.22,6.00,15.00,1.27,6.85,17.00,1.42,2.68,1.48,2.75,0.0,1.03,10.68,0.0886,0.1982,0.0422,0.0959,0.3367,0.2384,0,2010,9,25,5,1,248.11,0.819672,0.166667,0.066667,1.013850,4.285714,1.128237,3.0,4.0,0.886339,-0.113661,14.0,0.0,1.0,7.0,15.0,2.0,1.0,12.0,3.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,4.099784,0.096263,1.154701,0.000000,0,3,2010,6.0,0.0,10.0,0.0,1.0,7.0,9.0,-7.0,1,3.000000,0.000000,5.000000,0.000000,0.500000,3.500000,4.500000,-3.500000,0,0,1,2.0,0.0,0.0,0.0,0.0,2.0,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.753005,1.152007,1.400000,5.000000,1.000000,6.0,4.0,186.828186,3.012022,1.0,-0.180328,0.066667,-0.974928,-0.222521,-1.000000e+00,-1.836970e-16,-9.377521e-01,0.347305,0.217688,1.0,-0.849568
64675,E0,2010-11-07,Arsenal,Newcastle,1906.20,1710.25,9.0,9.0,7.0,7.0,0.0,1.0,A,0.0,1.0,A,11.0,7.0,8.0,2.0,16.0,12.0,12.0,2.0,3.0,1.0,1.0,0.0,1.29,5.50,11.00,1.30,6.11,14.04,1.45,2.61,1.50,2.79,0.0,1.05,8.49,0.2259,0.0406,0.0099,0.4871,0.0326,0.2038,0,2010,11,7,6,1,195.95,0.775194,0.181818,0.090909,1.044919,3.666667,1.154597,2.0,2.0,0.866103,-0.133897,5.0,4.0,4.0,3.0,15.0,5.0,5.0,7.0,2.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,1.762473,1.441365,0.577350,0.577350,0,3,2010,12.0,7.0,15.0,5.0,5.0,7.0,10.0,-2.0,1,2.400000,1.400000,3.000000,1.000000,1.000000,1.400000,2.000000,-0.400000,0,0,1,4.0,2.0,0.0,1.0,2.0,4.0,0.800000,0.400000,0.000000,0.200000,0.400000,0.800000,0.684285,1.114574,1.125000,1.500000,0.400000,5.0,2.0,134.085588,1.368569,2.0,-0.024806,-0.309091,-0.781831,0.623490,-5.000000e-01,8.660254e-01,9.884683e-01,0.151428,0.157193,0.0,-0.790073
65013,E0,2010-11-20,Arsenal,Tottenham,1887.19,1795.31,6.0,12.0,4.0,5.0,2.0,3.0,A,2.0,0.0,H,12.0,10.0,7.0,6.0,17.0,16.0,2.0,6.0,3.0,1.0,0.0,0.0,1.62,4.00,5.50,1.69,4.20,5.61,1.64,2.20,1.70,2.30,-0.8,1.82,2.06,0.1925,0.0099,0.0099,0.7242,0.0516,0.0120,0,2010,11,20,5,1,91.88,0.617284,0.250000,0.181818,1.153025,2.315789,1.251404,2.0,7.0,0.799102,-0.200898,3.0,4.0,2.0,7.0,9.0,5.0,6.0,9.0,0.0,0.0,0.0,0.0,1.0,2.0,3.0,0.0,0.881237,1.336757,1.000000,1.154701,0,3,2010,12.0,7.0,15.0,7.0,6.0,10.0,9.0,-3.0,1,2.000000,1.166667,2.500000,1.166667,1.000000,1.666667,1.500000,-0.500000,0,0,1,4.0,2.0,0.0,1.0,2.0,4.0,0.666667,0.333333,0.000000,0.166667,0.333333,0.666667,0.435466,1.051178,1.200000,1.153846,0.333333,5.0,7.0,40.010595,3.

In [14]:
# Categorical colums
cat_cols = ['Division', 'HomeTeam', 'AwayTeam']



# Numerical Colums
num_cols= ['Year',  'IsWeekend',
        'C_LTH', 'C_LTA', 'C_VHD', 'C_VAD', 'C_HTB', 'C_PHB',
        'HomeElo', 'AwayElo', 'EloDifference', 'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away', 'Form3Difference', 'Form5Difference',
        'GF3Home', 'GF3Away', 'GA3Home', 'GA3Away', 'GF5Home', 'GF5Away', 'GA5Home', 'GA5Away',
        'OddHome', 'OddDraw', 'OddAway', 'ImpliedProbHome', 'ImpliedProbDraw', 'ImpliedProbAway', 'BookmakerMargin',
        'HandiSize', 'HandiHome', 'HandiAway', 
        'MaxHome', 'MaxDraw', 'MaxAway', 'Over25', 'Under25', 'MaxOver25', 'MaxUnder25', 
        'WinStreakHome', 'WinStreakAway', 'DrawStreakHome', 'DrawStreakAway', 'DefeatStreakHome', 'DefeatStreakAway', 'H2HHomeWins', 'H2HAwayWins', 'GF_EMA3_Home','GF_EMA3_Away', 'GF3HomeSTD',
        'GF3AwaySTD', 'Season', 'PointsAcumHome', 'PointsAcumAway', 
        'GF_Total_Home', 'GF_Total_Away', 'GA_Total_Home', 'GA_Total_Away', 
        'GD_total_Home', 'GD_total_Away', 'PointMeanHome', 'PointMeanAway', 'ScoredGoalsMeanHome', 'ScoredGoalsMeanAway', 'ConcededGoalsMeanHome', 'ConcededGoalsMeanAway',
        'GoalsDifferenceMeanHome','GoalsDifferenceMeanAway','WinHomeAcum','WinAwayAcum','DrawHomeAcum','DrawAwayAcum','LossHomeAcum', 'LossAwayAcum', 'WinRateHome','WinRateAway','DrawRateHome',
        'DrawRateAway','LossRateHome','LossRateAway', 
        "OddsDifference", "EloRatio", "FormRatio", "GoalRateRatio", "WinRateDiff", "PointsDiff", "FormDiff", "Elo_ProbDiff", "FormOddsDiff", "StreakDiff", "BookieBiasHome", "DayOfWeek_sin", "DayOfWeek_cos", "Month_sin", "Month_cos", "Day_sin", "Day_cos",
        'OddSkew', 'FormVolatility', 'EloOddsGap']

# Numerical Colums without draw
num_cols_nd= ['Year',  'IsWeekend',
        'C_LTH', 'C_LTA', 'C_VHD', 'C_VAD', 'C_HTB', 'C_PHB',
        'HomeElo', 'AwayElo', 'EloDifference', 'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away', 'Form3Difference', 'Form5Difference',
        'GF3Home', 'GF3Away', 'GA3Home', 'GA3Away', 'GF5Home', 'GF5Away', 'GA5Home', 'GA5Away',
        'OddHome', 'OddAway', 'ImpliedProbHome', 'ImpliedProbAway', 'BookmakerMargin',
        'HandiSize', 'HandiHome', 'HandiAway', 
        'MaxHome', 'MaxAway', 'Over25', 'Under25', 'MaxOver25', 'MaxUnder25', 
        'WinStreakHome', 'WinStreakAway', 'DefeatStreakHome', 'DefeatStreakAway', 'H2HHomeWins', 'H2HAwayWins', 'GF_EMA3_Home','GF_EMA3_Away', 'GF3HomeSTD',
        'GF3AwaySTD', 'Season', 'PointsAcumHome', 'PointsAcumAway', 
        'GF_Total_Home', 'GF_Total_Away', 'GA_Total_Home', 'GA_Total_Away', 
        'GD_total_Home', 'GD_total_Away', 'PointMeanHome', 'PointMeanAway', 'ScoredGoalsMeanHome', 'ScoredGoalsMeanAway', 'ConcededGoalsMeanHome', 'ConcededGoalsMeanAway',
        'GoalsDifferenceMeanHome','GoalsDifferenceMeanAway','WinHomeAcum','WinAwayAcum','LossHomeAcum', 'LossAwayAcum', 'WinRateHome','WinRateAway',
        'LossRateHome','LossRateAway', 
        "OddsDifference", "EloRatio", "FormRatio", "GoalRateRatio", "WinRateDiff", "PointsDiff", "FormDiff", "Elo_ProbDiff", "FormOddsDiff", "StreakDiff", "BookieBiasHome", "DayOfWeek_sin", "DayOfWeek_cos", "Month_sin", "Month_cos", "Day_sin", "Day_cos",
        'OddSkew', 'FormVolatility', 'EloOddsGap']



num_pipe = Pipeline(steps=[ 
('imputer', SimpleImputer(strategy='median')),
('scaler', StandardScaler())
])


# Categorical pipe with OneHot
cat_pipe_onehot = Pipeline(steps=[
    ('one_hot_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Categorical pipe with Ordinal
cat_pipe_ordinal = Pipeline(steps=[
    ('ordinal_encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])


onehot_transformer = ColumnTransformer(transformers= [
    ('num_pipeline', num_pipe, num_cols),
    ('cat_pipeline', cat_pipe_onehot, cat_cols)], 
    remainder= 'passthrough',
    n_jobs= -1)

onehot_transformer_nd = ColumnTransformer(transformers= [
    ('num_pipeline', num_pipe, num_cols_nd),
    ('cat_pipeline', cat_pipe_onehot, cat_cols)], 
    remainder= 'passthrough',
    n_jobs= -1)


ordinal_transformer = ColumnTransformer(transformers= [
    ('num_pipeline', num_pipe, num_cols),
    ('cat_pipeline', cat_pipe_ordinal, cat_cols)], 
    remainder= 'passthrough',
    n_jobs= -1)

ordinal_transformer_nd = ColumnTransformer(transformers= [
    ('num_pipeline', num_pipe, num_cols_nd),
    ('cat_pipeline', cat_pipe_ordinal, cat_cols)], 
    remainder= 'passthrough',
    n_jobs= -1)

In [17]:
X_train_nd.replace([np.inf, -np.inf], 0, inplace=True)

In [18]:
pipe_rf_nd = Pipeline(steps=[
        ('preprocessor', onehot_transformer_nd),
        ('smote', SMOTE(random_state=42)),
        ('classifier', RandomForestClassifier(random_state=42))])

random_forest_params_nd = {
    'preprocessor__num_pipeline__scaler': [StandardScaler(), MinMaxScaler(), None],
    'preprocessor': [onehot_transformer_nd, ordinal_transformer_nd],
    "classifier__n_estimators": [200, 300, 500, 1000],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4],
    'classifier__max_features': ['sqrt', 'log2', None],
    "classifier__max_depth": [3, 5, 7, 15]}

random_forest_RS_nd = RandomizedSearchCV(estimator = pipe_rf_nd,
                    param_distributions=  random_forest_params_nd,
                    cv = 5,
                    verbose=2,
                    n_jobs=-1, 
                    scoring= 'accuracy',
                    random_state=42)

random_forest_RS_nd.fit(X_train_nd, y_train_nd)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\Renan Muniz\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(n_jobs=-1,
                                                                remainder='passthrough',
                                                                transformers=[('num_pipeline',
                                                                               Pipeline(steps=[('imputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('scaler',
                                                                                                StandardScaler())]),
                                                                               ['Year',
                                                                                'IsWeekend',
                                                                                'C_LTH',
                                                                                'C_LTA',
                                                                                'C_VHD',
                                                                                'C_VAD',
                                                                                'C_HTB',
                                                                                'C_PHB',
                                                                                'HomeElo',
                                                                                'AwayElo',
                                                                                'EloDifference',
                                                                                'Form3H...
                                                                                           'OddAway',
                                                                                           'ImpliedProbHome',
                                                                                           'ImpliedProbAway',
                                                                                           'BookmakerMargin', ...]),
                                                                                         ('cat_pipeline',
                                                                                          Pipeline(steps=[('ordinal_encoder',
                                                                                                           OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                                                          unknown_value=-1))]),
                                                                                          ['Division',
                                                                                           'HomeTeam',
                                                                                           'AwayTeam'])])],
                                        'preprocessor__num_pipeline__scaler': [StandardScaler(),
                                                                               MinMaxScaler(),
                                                                               None]},
                   random_state=42, scoring='accuracy', verbose=2)